In [ ]:
import os
#import omnisafe
from omnisafe import Evaluator
from gym_navigation.envs.navigation_goal_safe import NavigationGoalSafe
from omnisafe.envs.wrapper import TimeLimit
import torch



save_dir = "runs/PPOLag-{NavigationGoalSafe-v0}/seed-000-2025-05-03-10-29-25"

available_files = [f for f in os.listdir(save_dir)]
print(f"Available files: {available_files}")
model_name = "epoch-0.pt"

evaluator = Evaluator() 

evaluator.load_saved(
    save_dir=save_dir,
    model_name='epoch-500.pt',
)

evaluator._env = TimeLimit(evaluator._env, device=torch.device('cpu'), time_limit=1000)

evaluator.evaluate(num_episodes=1000)

In [ ]:
import omnisafe
import os
from gym_navigation.envs.navigation_goal_safe import NavigationGoalSafe
from omnisafe.envs.wrapper import TimeLimit
import torch


save_dir = "runs/PPOLag-{NavigationGoalSafe-v0}/seed-000-2025-05-03-19-37-37"

evaluator = omnisafe.Evaluator(render_mode='rgb_array')

save_replay_path = 'videos/' + save_dir.split('/')[1] + '/' + save_dir.split('/')[2] + '/'

# Load the saved model and render episodes
for item in os.scandir(os.path.join(save_dir, 'torch_save')):
    if item.is_file() and item.name.split('.')[-1] == 'pt':
        if item.name == 'epoch-500.pt': # only load the last checkpoint
            evaluator.load_saved(
                save_dir= save_dir,
                model_name=item.name,
                width=256,
                height=256,
                render_mode='rgb_array'
            )
            # limit the environment to 1000 steps
            evaluator._env = TimeLimit(evaluator._env, device=torch.device('cpu'), time_limit=1000)
            # save videos
            evaluator.render(num_episodes=10, save_replay_path=save_replay_path + "/" + item.name.split('.')[0])